# Convolutional Neural Network

### Importing the libraries

In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing import image
import numpy as np
import os

In [2]:
train_dir = "dataset/training_set"
test_dir = "dataset/test_set"
prediction_dir = "dataset/single_prediction"

train_datagen = ImageDataGenerator(rescale=1.0/255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1.0/255)

## Part 1 - Data Preprocessing

### Preprocessing the Training set

In [3]:
train_set = train_datagen.flow_from_directory(train_dir,
                                              target_size=(64, 64),
                                              batch_size=32,
                                              class_mode="binary")

Found 8000 images belonging to 2 classes.


### Preprocessing the Test set

In [4]:
test_set = test_datagen.flow_from_directory(test_dir,
                                            target_size=(64, 64),
                                            batch_size=32,
                                            class_mode="binary")

Found 2000 images belonging to 2 classes.


## Part 2 - Building the CNN

### Initialising the CNN

In [5]:
input_layer = tf.keras.Input(shape=(64, 64, 3))

### Step 1 - Convolution

In [6]:
x = tf.keras.layers.Conv2D(32, (3, 3), activation='relu')(input_layer)

2024-12-14 17:14:04.258842: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1
2024-12-14 17:14:04.258964: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2024-12-14 17:14:04.258989: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2024-12-14 17:14:04.259341: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-12-14 17:14:04.259408: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


### Step 2 - Pooling

In [7]:
x = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(x)

### Adding a second convolutional layer

In [8]:
x = tf.keras.layers.Conv2D(64, (3, 3), activation='relu')(x)
x = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(x)

x = tf.keras.layers.Conv2D(128, (3, 3), activation='relu')(x)
x = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(x)

### Step 3 - Flattening

In [9]:
x = tf.keras.layers.Flatten()(x)

### Step 4 - Full Connection

In [10]:
x = tf.keras.layers.Dense(128, activation='relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)

### Step 5 - Output Layer

In [11]:
output_layer = tf.keras.layers.Dense(1, activation='sigmoid')(x)

## Part 3 - Training the CNN

### Compiling the CNN

In [12]:
model = tf.keras.Model(inputs=input_layer, outputs=output_layer)
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

### Training the CNN on the Training set and evaluating it on the Test set

In [13]:
model.fit(train_set, epochs=20, validation_data=test_set)
test_loss, test_accuracy = model.evaluate(test_set)
print(f"Test Accuracy: {test_accuracy:.2f}")

/opt/anaconda3/envs/tensorflow/lib/python3.11/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20


2024-12-14 17:14:16.739578: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


249/250 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - accuracy: 0.5303 - loss: 0.6917

/opt/anaconda3/envs/tensorflow/lib/python3.11/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


250/250 ━━━━━━━━━━━━━━━━━━━━ 15s 44ms/step - accuracy: 0.5306 - loss: 0.6916 - val_accuracy: 0.6415 - val_loss: 0.6511
Epoch 2/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 11s 44ms/step - accuracy: 0.6398 - loss: 0.6416 - val_accuracy: 0.6350 - val_loss: 0.6205
Epoch 3/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 12s 48ms/step - accuracy: 0.6911 - loss: 0.5879 - val_accuracy: 0.7470 - val_loss: 0.5215
Epoch 4/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 11s 44ms/step - accuracy: 0.7386 - loss: 0.5353 - val_accuracy: 0.7035 - val_loss: 0.5747
Epoch 5/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 11s 45ms/step - accuracy: 0.7523 - loss: 0.5085 - val_accuracy: 0.7400 - val_loss: 0.5529
Epoch 6/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 11s 44ms/step - accuracy: 0.7713 - loss: 0.4860 - val_accuracy: 0.7970 - val_loss: 0.4424
Epoch 7/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 11s 43ms/step - accuracy: 0.7895 - loss: 0.4627 - val_accuracy: 0.8030 - val_loss: 0.4265
Epoch 8/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 11s 42ms/step - accuracy: 0.8094 - loss: 0.4243 - val_accurac

## Part 4 - Making a single prediction

In [14]:
from keras.src.utils import load_img, img_to_array


def predict_single_image(image_path):
    image = load_img(image_path, target_size=(64, 64))
    image_array = img_to_array(image) / 255.0
    image_array = np.expand_dims(image_array, axis=0)
    prediction = model.predict(image_array)
    return "Dog" if prediction[0][0] > 0.5 else "Cat"



In [15]:
for filename in os.listdir(prediction_dir):
    if filename.lower().endswith(('png', 'jpg', 'jpeg')):
        file_path = os.path.join(prediction_dir, filename)
        result = predict_single_image(file_path)
        print(f"Prediction for {filename}: {result}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Prediction for cat_or_dog_1.jpg: Dog
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
Prediction for cat_or_dog_2.jpg: Cat


## github repository link
- https://github.com/jerry0chou/CNN-for-Image-Classification